<a href="https://colab.research.google.com/github/Chediak/common-master-ai/blob/main/prodesan_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber sentence-transformers faiss-cpu spacy
!python -m spacy download pt_core_news_sm
!pip install fastapi uvicorn
!pip install python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 84.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pdfplumber
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import spacy
import json
import re
from datetime import datetime

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        pages = [page.extract_text() for page in pdf.pages]
    return pages

def preprocess_and_split_text(pages):
    news_items = [item.strip() for page in pages for item in page.split("\n\n") if item.strip()]
    return news_items

def generate_embeddings(news_items, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(news_items)
    return embeddings, model

def create_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

def add_metadata(news_items):
    metadata = [{"id": i, "content": news_items[i], "length": len(news_items[i])} for i in range(len(news_items))]
    return metadata

def extract_entities(news_items):
    nlp = spacy.load("pt_core_news_sm")
    extracted_data = []
    for idx, news in enumerate(news_items):
        doc = nlp(news)
        entities = {"id": idx, "content": news, "dates": [], "names": [], "organizations": []}

        for ent in doc.ents:
            if ent.label_ == "DATE":
                entities["dates"].append(ent.text)
            elif ent.label_ == "PERSON":
                entities["names"].append(ent.text)
            elif ent.label_ == "ORG":
                entities["organizations"].append(ent.text)

        cnpj_matches = re.findall(r"\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}", news)
        entities["cnpjs"] = cnpj_matches
        extracted_data.append(entities)
    return extracted_data

def create_structured_index(entities):
    index = {"dates": {}, "names": {}, "organizations": {}, "cnpjs": {}}
    for item in entities:
        for date in item["dates"]:
            index["dates"].setdefault(date, []).append(item)
        for name in item["names"]:
            index["names"].setdefault(name, []).append(item)
        for org in item["organizations"]:
            index["organizations"].setdefault(org, []).append(item)
        for cnpj in item["cnpjs"]:
            index["cnpjs"].setdefault(cnpj, []).append(item)
    return index

def query_index(index, query_type, query_value):
    if query_type in index:
        return index[query_type].get(query_value, [])
    return []

def optimize_layout(metadata, top_k=5):
    sorted_news = sorted(metadata, key=lambda x: x["length"], reverse=True)
    layout = []
    for i, item in enumerate(sorted_news):
        layout.append({
            "type": "news",
            "content": item["content"],
            "page": i // top_k + 1,
            "position": i % top_k
        })
        if (i + 1) % 3 == 0:
            layout.append({
                "type": "advertisement",
                "content": "Ad Placeholder",
                "page": (i + 1) // top_k + 1,
                "position": "bottom"
            })
    return layout

def main():
    pdf_path = "/content/RHOAI _ Prodesp - Diário Oficial.pdf"

    print("Extracting text from PDF...")
    pages = extract_text_from_pdf(pdf_path)

    print("Preprocessing and splitting text...")
    news_items = preprocess_and_split_text(pages)

    print("Generating embeddings...")
    embeddings, model = generate_embeddings(news_items)

    print("Creating FAISS index...")
    index = create_faiss_index(np.array(embeddings))

    print("Adding metadata...")
    metadata = add_metadata(news_items)

    print("Extracting entities...")
    entities = extract_entities(news_items)

    print("Creating structured index...")
    structured_index = create_structured_index(entities)

    print("Querying by date '13/11/2024'...")
    date_results = query_index(structured_index, "dates", "13/11/2024")
    print(json.dumps(date_results, indent=2))

    print("Querying by organization 'ARAUJO E REPLANDE LTDA'...")
    org_results = query_index(structured_index, "organizations", "ARAUJO E REPLANDE LTDA")
    print(json.dumps(org_results, indent=2))

    print("Optimizing layout...")
    layout = optimize_layout(metadata)
    print(json.dumps(layout, indent=2))

if __name__ == "__main__":
    main()


Extracting text from PDF...
Preprocessing and splitting text...
Generating embeddings...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating FAISS index...
Adding metadata...
Extracting entities...
Creating structured index...
Querying by date '13/11/2024'...
[]
Querying by organization 'ARAUJO E REPLANDE LTDA'...
[]
Optimizing layout...
[
  {
    "type": "news",
    "content": "EstruturadoJSONderetornoesperadoqueser\u00e1geradopelaRedHat\n{ \"pdf_data\":\n[\n{\"id\": \u201cf97fb8c9-60d6-4fed-a82d-1cdf4e8be0bc\u201d ,\"x\":50,\"y\":300,\"pagina\":1,\"tipo\":\"mat\u00e9ria\"},\n{\u201cid\u201d:\u201c\",\"x\":100,\"y\":550,\"pagina\":1,\"tipo\":\"titulo\"},\n{\u201cid\u201d:\u201cd2a7e6bd-5d6d-4e1c-a747-a0ae5f1499da\",\"x\":100,\"y\":550,\"pagina\":1,\"tipo\":\"mat\u00e9ria\"}\n{\u201cid\u201d:\u201c\",\"x\":100,\"y\":550,\"pagina\":1,\"tipo\":\"calhau-1\"}\n]\n}\n\u2014--------\nDi\u00e1riooficial:tamanhoA3\n-Alinhamentodasmat\u00e9riascomamargemsuperiorquandonaprimeiralinhainterpretando\n\"Alinhamento\"comodist\u00e2nciadamargemsuperior:17mm.\n-Alinhamentodasmat\u00e9riascomamargeminferiorquandonaultimalinha,interp

In [4]:
import pdfplumber
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        pages = [page.extract_text() for page in pdf.pages]
    return pages

def preprocess_and_split_text(pages):
    news_items = [item.strip() for page in pages for item in page.split("\n\n") if item.strip()]
    return news_items

def generate_embeddings(news_items, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(news_items)
    return embeddings, model

def create_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

def add_metadata(news_items):
    metadata = [{"id": i, "content": news_items[i], "length": len(news_items[i])} for i in range(len(news_items))]
    return metadata

def search_similar_news(query, model, index, metadata, top_k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = [
        {"content": metadata[idx]["content"], "distance": float(dist), "length": metadata[idx]["length"]}
        for dist, idx in zip(distances[0], indices[0])
    ]
    return results

def optimize_layout(metadata, top_k=5):
    # Sort news items by length (longer news first)
    sorted_news = sorted(metadata, key=lambda x: x["length"], reverse=True)

    layout = []
    for i, item in enumerate(sorted_news):
        layout.append({"type": "news", "content": item["content"], "page": i // top_k + 1, "position": i % top_k})
        if (i + 1) % 3 == 0:
            layout.append({"type": "advertisement", "content": "Ad Placeholder", "page": (i + 1) // top_k + 1, "position": "bottom"})

    return layout

def main():
    pdf_path = "/content/RHOAI _ Prodesp - Diário Oficial.pdf"

    print("Extracting text from PDF...")
    pages = extract_text_from_pdf(pdf_path)

    print("Preprocessing and splitting text...")
    news_items = preprocess_and_split_text(pages)

    print("Generating embeddings...")
    embeddings, model = generate_embeddings(news_items)

    print("Creating FAISS index...")
    index = create_faiss_index(np.array(embeddings))

    print("Adding metadata...")
    metadata = add_metadata(news_items)

    query = "example query about a topic"
    print("Searching for similar news...")
    results = search_similar_news(query, model, index, metadata, top_k=5)

    print("Search Results:")
    print(json.dumps(results, indent=2))

    print("Optimizing layout...")
    layout = optimize_layout(metadata)

    print("Layout:")
    print(json.dumps(layout, indent=2))

if __name__ == "__main__":
    main()


Extracting text from PDF...
Preprocessing and splitting text...
Generating embeddings...
Creating FAISS index...
Adding metadata...
Searching for similar news...
Search Results:
[
  {
    "content": "-Justificarasmat\u00e9riasqueest\u00e3odentrodamesmacolunaautomaticamente\n10.Entreasse\u00e7\u00f5esdemat\u00e9ria,incluirumtextoautomaticamentedaquebradese\u00e7\u00e3o\n4.Extra\u00e7\u00e3o\na. Arquitetura\n#TODO\n7",
    "distance": 1.64579439163208,
    "length": 168
  },
  {
    "content": "EstruturadoJSONderetornoesperadoqueser\u00e1geradopelaRedHat\n{ \"pdf_data\":\n[\n{\"id\": \u201cf97fb8c9-60d6-4fed-a82d-1cdf4e8be0bc\u201d ,\"x\":50,\"y\":300,\"pagina\":1,\"tipo\":\"mat\u00e9ria\"},\n{\u201cid\u201d:\u201c\",\"x\":100,\"y\":550,\"pagina\":1,\"tipo\":\"titulo\"},\n{\u201cid\u201d:\u201cd2a7e6bd-5d6d-4e1c-a747-a0ae5f1499da\",\"x\":100,\"y\":550,\"pagina\":1,\"tipo\":\"mat\u00e9ria\"}\n{\u201cid\u201d:\u201c\",\"x\":100,\"y\":550,\"pagina\":1,\"tipo\":\"calhau-1\"}\n]\n}\n\u2014---

In [5]:
import json

input_data = {
    "pdf_data": [
        {"id": "f97fb8c9-60d6-4fed-a82d-1cdf4e8be0bc", "altura": 125, "largura": 35, "agrupamento": 1},
        {"id": "f97fb8c9-60d6-4fed-a82d-1cdf4e8be0bc", "altura": 70, "largura": 40, "agrupamento": 2}
    ]
}

margin_top = 17  # mm
margin_bottom = 13  # mm
margin_left = 13  # mm
margin_right = 13  # mm
min_spacing_mm = 5  # mm
page_height_mm = 420
ad_sizes = [(81, 100), (250, 210)]

def generate_output(input_data):
    output_data = {"pdf_data": []}
    page = 1
    current_y = margin_top

    for item in input_data["pdf_data"]:
        content_type = "matéria" if item["agrupamento"] == 1 else "titulo"
        output_data["pdf_data"].append({
            "id": item["id"],
            "x": margin_left,
            "y": current_y,
            "pagina": page,
            "tipo": content_type
        })
        current_y += item["altura"] + min_spacing_mm

        if current_y + margin_bottom > page_height_mm:
            page += 1
            current_y = margin_top

        for ad_width, ad_height in ad_sizes:
            if current_y + ad_height + margin_bottom <= page_height_mm:
                output_data["pdf_data"].append({
                    "id": "calhau-placeholder",
                    "x": margin_left,
                    "y": current_y,
                    "pagina": page,
                    "tipo": f"calhau-{ad_width}x{ad_height}"
                })
                current_y += ad_height + min_spacing_mm
                break

    return output_data

structured_output = generate_output(input_data)

output_path = "structured_output.json"
with open(output_path, "w") as f:
    json.dump(structured_output, f, indent=2)

print(f"Structured JSON output saved to {output_path}")
structured_output


Structured JSON output saved to structured_output.json


{'pdf_data': [{'id': 'f97fb8c9-60d6-4fed-a82d-1cdf4e8be0bc',
   'x': 13,
   'y': 17,
   'pagina': 1,
   'tipo': 'matéria'},
  {'id': 'calhau-placeholder',
   'x': 13,
   'y': 147,
   'pagina': 1,
   'tipo': 'calhau-81x100'},
  {'id': 'f97fb8c9-60d6-4fed-a82d-1cdf4e8be0bc',
   'x': 13,
   'y': 252,
   'pagina': 1,
   'tipo': 'titulo'}]}

In [6]:
import pdfplumber
import json

margin_top = 17  # mm
margin_bottom = 13  # mm
margin_left = 13  # mm
margin_right = 13  # mm
min_spacing_mm = 5  # mm
page_height_mm = 420
ad_sizes = [(81, 100), (250, 210)]

def extract_pdf_data(pdf_path):
    extracted_data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            text = page.extract_text()
            if not text:
                continue
            # Split text into blocks for processing
            for idx, block in enumerate(text.split("\n\n")):
                extracted_data.append({
                    "id": f"block-{page_number}-{idx}",
                    "content": block.strip(),
                    "altura": len(block) * 0.1,
                    "largura": 200,
                    "agrupamento": 1 if idx % 2 == 0 else 2
                })
    return extracted_data

def generate_output(pdf_data):
    output_data = {"pdf_data": []}
    page = 1
    current_y = margin_top

    for item in pdf_data:
        content_type = "matéria" if item["agrupamento"] == 1 else "titulo"
        output_data["pdf_data"].append({
            "id": item["id"],
            "x": margin_left,
            "y": current_y,
            "pagina": page,
            "tipo": content_type
        })
        current_y += item["altura"] + min_spacing_mm

        if current_y + margin_bottom > page_height_mm:
            page += 1
            current_y = margin_top

        for ad_width, ad_height in ad_sizes:
            if current_y + ad_height + margin_bottom <= page_height_mm:
                output_data["pdf_data"].append({
                    "id": "calhau-placeholder",
                    "x": margin_left,
                    "y": current_y,
                    "pagina": page,
                    "tipo": f"calhau-{ad_width}x{ad_height}"
                })
                current_y += ad_height + min_spacing_mm
                break

    return output_data

pdf_path = "/content/RHOAI _ Prodesp - Diário Oficial.pdf"

print("Extracting data from the PDF...")
pdf_data = extract_pdf_data(pdf_path)

print("Generating structured JSON output...")
structured_output = generate_output(pdf_data)

output_path = "structured_output.json"
with open(output_path, "w") as f:
    json.dump(structured_output, f, indent=2)

print(f"Structured JSON output saved to {output_path}")
structured_output


Extracting data from the PDF...
Generating structured JSON output...
Structured JSON output saved to structured_output.json


{'pdf_data': [{'id': 'block-1-0',
   'x': 13,
   'y': 17,
   'pagina': 1,
   'tipo': 'matéria'},
  {'id': 'calhau-placeholder',
   'x': 13,
   'y': 32.1,
   'pagina': 1,
   'tipo': 'calhau-81x100'},
  {'id': 'block-2-0', 'x': 13, 'y': 137.1, 'pagina': 1, 'tipo': 'matéria'},
  {'id': 'calhau-placeholder',
   'x': 13,
   'y': 159.4,
   'pagina': 1,
   'tipo': 'calhau-81x100'},
  {'id': 'block-3-0', 'x': 13, 'y': 264.4, 'pagina': 1, 'tipo': 'matéria'},
  {'id': 'block-4-0',
   'x': 13,
   'y': 318.79999999999995,
   'pagina': 1,
   'tipo': 'matéria'},
  {'id': 'block-5-0',
   'x': 13,
   'y': 355.59999999999997,
   'pagina': 1,
   'tipo': 'matéria'},
  {'id': 'calhau-placeholder',
   'x': 13,
   'y': 17,
   'pagina': 2,
   'tipo': 'calhau-81x100'},
  {'id': 'block-6-0', 'x': 13, 'y': 122, 'pagina': 2, 'tipo': 'matéria'},
  {'id': 'calhau-placeholder',
   'x': 13,
   'y': 170.9,
   'pagina': 2,
   'tipo': 'calhau-81x100'},
  {'id': 'block-7-0', 'x': 13, 'y': 275.9, 'pagina': 2, 'tipo': 'ma

In [7]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import pdfplumber
import json
import os

app = FastAPI()

margin_top = 17  # mm
margin_bottom = 13  # mm
margin_left = 13  # mm
margin_right = 13  # mm
min_spacing_mm = 5  # mm
page_height_mm = 420
ad_sizes = [(81, 100), (250, 210)]

def extract_pdf_data(pdf_path):
    extracted_data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            text = page.extract_text()
            if not text:
                continue
            for idx, block in enumerate(text.split("\n\n")):
                extracted_data.append({
                    "id": f"block-{page_number}-{idx}",
                    "content": block.strip(),
                    "altura": len(block) * 0.1,
                    "largura": 200,
                    "agrupamento": 1 if idx % 2 == 0 else 2
                })
    return extracted_data

def generate_output(pdf_data):
    output_data = {"pdf_data": []}
    page = 1
    current_y = margin_top

    for item in pdf_data:
        content_type = "matéria" if item["agrupamento"] == 1 else "titulo"
        output_data["pdf_data"].append({
            "id": item["id"],
            "x": margin_left,
            "y": current_y,
            "pagina": page,
            "tipo": content_type
        })
        current_y += item["altura"] + min_spacing_mm

        if current_y + margin_bottom > page_height_mm:
            page += 1
            current_y = margin_top

        for ad_width, ad_height in ad_sizes:
            if current_y + ad_height + margin_bottom <= page_height_mm:
                output_data["pdf_data"].append({
                    "id": "calhau-placeholder",
                    "x": margin_left,
                    "y": current_y,
                    "pagina": page,
                    "tipo": f"calhau-{ad_width}x{ad_height}"
                })
                current_y += ad_height + min_spacing_mm
                break

    return output_data

@app.post("/upload-pdf/")
async def upload_pdf(file: UploadFile = File(...)):
    pdf_path = f"./{file.filename}"
    with open(pdf_path, "wb") as f:
        f.write(await file.read())

    try:
        pdf_data = extract_pdf_data(pdf_path)

        structured_output = generate_output(pdf_data)

        output_path = f"./{os.path.splitext(file.filename)[0]}_output.json"
        with open(output_path, "w") as f:
            json.dump(structured_output, f, indent=2)

        return JSONResponse(content=structured_output)

    except Exception as e:
        return {"error": str(e)}

    finally:
        if os.path.exists(pdf_path):
            os.remove(pdf_path)

# To run the API server:
# Use the command: uvicorn <filename>:app --reload


In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import pdfplumber
import json
import os
import nest_asyncio
import uvicorn

# Initialize FastAPI app
app = FastAPI()

# Constants for layout rules
margin_top = 17  # mm
margin_bottom = 13  # mm
margin_left = 13  # mm
margin_right = 13  # mm
min_spacing_mm = 5  # mm
page_height_mm = 420  # Approx height of A3 in mm
ad_sizes = [(81, 100), (250, 210)]  # Example sizes in mm

# Extract text and preprocess data from PDF
def extract_pdf_data(pdf_path):
    extracted_data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            text = page.extract_text()
            if not text:
                continue
            # Split text into blocks for processing
            for idx, block in enumerate(text.split("\n\n")):
                extracted_data.append({
                    "id": f"block-{page_number}-{idx}",
                    "content": block.strip(),
                    "altura": len(block) * 0.1,  # Mock height based on content length
                    "largura": 200,  # Arbitrary width
                    "agrupamento": 1 if idx % 2 == 0 else 2  # Alternate grouping
                })
    return extracted_data

# Generate the expected JSON output with alignment and spacing rules
def generate_output(pdf_data):
    output_data = {"pdf_data": []}
    page = 1
    current_y = margin_top  # Start at the top margin

    for item in pdf_data:
        # Add an entry for the main content
        content_type = "matéria" if item["agrupamento"] == 1 else "titulo"
        output_data["pdf_data"].append({
            "id": item["id"],
            "x": margin_left,  # Always align to left margin
            "y": current_y,
            "pagina": page,
            "tipo": content_type
        })
        # Update Y position considering the height and spacing
        current_y += item["altura"] + min_spacing_mm

        # Check if a new page is needed
        if current_y + margin_bottom > page_height_mm:
            page += 1
            current_y = margin_top

        # Add calhau (ad placeholder) if space permits
        for ad_width, ad_height in ad_sizes:
            if current_y + ad_height + margin_bottom <= page_height_mm:
                output_data["pdf_data"].append({
                    "id": "calhau-placeholder",
                    "x": margin_left,
                    "y": current_y,
                    "pagina": page,
                    "tipo": f"calhau-{ad_width}x{ad_height}"
                })
                current_y += ad_height + min_spacing_mm
                break

    return output_data

@app.post("/upload-pdf/")
async def upload_pdf(file: UploadFile = File(...)):
    # Save the uploaded file
    pdf_path = f"./{file.filename}"
    with open(pdf_path, "wb") as f:
        f.write(await file.read())

    try:
        # Extract data from the PDF
        pdf_data = extract_pdf_data(pdf_path)

        # Generate the structured output
        structured_output = generate_output(pdf_data)

        # Save the structured JSON output
        output_path = f"./{os.path.splitext(file.filename)[0]}_output.json"
        with open(output_path, "w") as f:
            json.dump(structured_output, f, indent=2)

        # Return the structured output as response
        return JSONResponse(content=structured_output)

    except Exception as e:
        return {"error": str(e)}

    finally:
        # Clean up: Remove the uploaded file
        if os.path.exists(pdf_path):
            os.remove(pdf_path)

# Run the server in Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()  # Allow nested event loops in Jupyter

# Start the FastAPI server
print("Starting FastAPI server...")
uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [190]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Starting FastAPI server...


In [ ]:
import requests

url = "http://127.0.0.1:8000/upload-pdf/"
files = {"file": open("RHOAI _ Prodesp - Diário Oficial.pdf", "rb")}
response = requests.post(url, files=files)

print(response.json())